In [8]:
import re
import csv
from collections import Counter

ip_pattern = re.compile(r'^(\d+\.\d+\.\d+\.\d+)')
endpoint_pattern = re.compile(r' "(GET|POST|PUT|DELETE) ([^ ]+) HTTP')
status_pattern = re.compile(r' " (\d{3}) ')

failed_login_threshold = 10

log_file = '/content/sample.log'
output_file = 'log_analysis_results.csv'

def parse_log_file(log_file):
    ip_requests = Counter()
    endpoint_requests = Counter()
    failed_login_attempts = Counter()

    with open(log_file, 'r') as file:
        for line in file:
            ip = ip_pattern.search(line)
            if ip:
                ip_requests[ip.group(1)] += 1

                endpoint_match = endpoint_pattern.search(line)
                if endpoint_match:
                    endpoint_requests[endpoint_match.group(2)] += 1

                status_match = status_pattern.search(line)
                if status_match and status_match.group(1) == '401':
                    failed_login_attempts[ip.group(1)] += 1

    return ip_requests, endpoint_requests, failed_login_attempts

def print_terminal_report(ip_requests, endpoint_requests, failed_login_attempts):
    print("\n--- Log Analysis Report ---")

    print("\nRequests per IP Address:")
    for ip, count in ip_requests.most_common():
        print(f"{ip:<20} {count}")

    print("\nMost Frequently Accessed Endpoint:")
    most_accessed_endpoint = endpoint_requests.most_common(1)
    if most_accessed_endpoint:
        endpoint, count = most_accessed_endpoint[0]
        print(f"{endpoint} (Accessed {count} times)")

    print("\nSuspicious Activity Detected:")
    suspicious_ips = {ip: count for ip, count in failed_login_attempts.items() if count > failed_login_threshold}
    if suspicious_ips:
        for ip, count in suspicious_ips.items():
            print(f"{ip:<20} {count}")
    else:
        print("No suspicious activity detected.")

def save_csv_report(ip_requests, endpoint_requests, failed_login_attempts):
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        writer.writerow(["Requests per IP Address"])
        writer.writerow(["IP Address", "Request Count"])
        for ip, count in ip_requests.most_common():
            writer.writerow([ip, count])

        writer.writerow([])
        writer.writerow(["Most Accessed Endpoint"])
        writer.writerow(["Endpoint", "Access Count"])
        most_accessed_endpoint = endpoint_requests.most_common(1)
        if most_accessed_endpoint:
            endpoint, count = most_accessed_endpoint[0]
            writer.writerow([endpoint, count])

        writer.writerow([])
        writer.writerow(["Suspicious Activity"])
        writer.writerow(["IP Address", "Failed Login Attempts"])
        suspicious_ips = {ip: count for ip, count in failed_login_attempts.items() if count > failed_login_threshold}
        for ip, count in suspicious_ips.items():
            writer.writerow([ip, count])

    print(f"\nResults saved to {output_file}")

def main():
    ip_requests, endpoint_requests, failed_login_attempts = parse_log_file(log_file)
    print_terminal_report(ip_requests, endpoint_requests, failed_login_attempts)
    save_csv_report(ip_requests, endpoint_requests, failed_login_attempts)

if __name__ == "__main__":
    main()



--- Log Analysis Report ---

Requests per IP Address:
203.0.113.5          8
198.51.100.23        8
192.168.1.1          7
10.0.0.2             6
192.168.1.100        5

Most Frequently Accessed Endpoint:
/login (Accessed 13 times)

Suspicious Activity Detected:
No suspicious activity detected.

Results saved to log_analysis_results.csv
